In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

In [3]:
#Step-1
#creating 30 samples from the whole boston data points.
col=np.arange(13)
col_set=[]
X_train=[]
X_test=[]
y_train=[]
y_test=[]
col_train_set=[]
for i in range(30):
    #randomly choosing different feature for each sample
    col_train, col_ignore= train_test_split(col, test_size=0.50,random_state=2*i)
    col_train=np.sort(col_train)
    col_train_set.append(col_train)
    X=x[:,col_train]
    X_tr0, X_te0, y_tr0, y_te0 = train_test_split(X, y, test_size=0.40,random_state=3*i) #60% of x data
    X_tr1, X_te1, y_tr1, y_te1 = train_test_split(X_tr0, y_tr0, test_size=0.33,random_state=2*i) #taking 40%of data from x_train(60% of x)
    #addiing up the 60% and 40% values
    X_tr_final=np.vstack((X_tr0,X_tr1))
    X_te_final=np.vstack((X_te0,X_te1))
    y_tr_final=np.hstack((y_tr0,y_tr1))
    y_te_final=np.hstack((y_te0,y_te1))
    #Corpus of 30 samples
    X_train.append(X_tr_final)
    X_test.append(X_te_final)
    y_train.append(y_tr_final)
    y_test.append(y_te_final)

In [4]:
#Step-2
from sklearn.tree import DecisionTreeRegressor
clf=DecisionTreeRegressor()
y_train_pred=[]
y_test_pred=[]
y_pred=[]
#y_pred corpus
for i in range(30):
    y_total_pred=[]
    clf.fit(X_train[i], y_train[i])
    y_pred.append(clf.predict(X_train[i]))
#computing Average y_pre value    
y_sum=y_pred[0]
for j in range(len(y_pred)):
    y_sum=+np.add(y_sum,y_pred[j])
y_pred_avg=np.true_divide(y_sum, 30)
#computing MSE:
y_mse=np.subtract(y,y_pred_avg)#y-y_pred
y_mse_sqr=np.square(y_mse)#squaring the diff
y_mse_final=np.true_divide(y_mse_sqr, 506)
print(len(y_mse_final))

506


In [5]:
#Step-3:
flag=[]
X_trains=np.asarray(X_train)
for i in range(30):
    for j in range(len(x)):
        #p=x[:,col_train_set[i]][j].tolist()
        if x[:,col_train_set[i]][j] in X_trains[i]:
            flag.append(1)
        else:
            flag.append(1)            
print((flag))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 